# District Cleaning — Punjab

This notebook standardizes **district names** for this state across:
- Enrolment data
- Demographic update data
- Biometric update data

**All data is saved back to the same cleaned files.**

In [2]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

CLEAN_DIR = Path("../../data/processed/cleaned")

enrol_df = pd.read_csv(CLEAN_DIR / "enrolment_clean.csv")
demo_df  = pd.read_csv(CLEAN_DIR / "demographic_clean.csv")
bio_df   = pd.read_csv(CLEAN_DIR / "biometric_clean.csv")

for df in [enrol_df, demo_df, bio_df]:
    df["state"] = df["state"].astype(str).str.strip().str.title()
    df["district"] = df["district"].astype(str).str.strip().str.title()

print("✅ All datasets loaded and normalized (Title Case)")


✅ All datasets loaded and normalized (Title Case)


In [6]:
STATE_NAME = "Punjab"

districts = sorted(
    set(
        enrol_df.loc[enrol_df["state"] == STATE_NAME, "district"].dropna()
        .tolist()
    )
)

print(f"State: {STATE_NAME}")
print(f"Number of unique districts: {len(districts)}")

pd.DataFrame(
    {"District Name": districts}
)


State: Punjab
Number of unique districts: 23


,District Name
0,Amritsar
1,Barnala
2,Bathinda
3,Faridkot
4,Fatehgarh Sahib
5,Fazilka
6,Ferozepur
7,Gurdaspur
8,Hoshiarpur
9,Jalandhar


## District Mapping

Add mappings in **Title Case only**.

Format:
```python
DISTRICT_MAPPING = {
    "Correct District": ["Wrong Name 1", "Wrong Name 2"],
}
```

In [4]:
DISTRICT_MAPPING = {
    # "Correct District": ["Wrong Variant 1", "Wrong Variant 2"]
    "Ferozepur": ["Firozpur"],
    "Sri Muktsar Sahib": ["Muktsar"],
    "Shaheed Bhagat Singh Nagar": ["Nawanshahr"],
    "Sahibzada Ajit Singh Nagar": ["S.A.S Nagar","S.A.S Nagar(Mohali)","Sas Nagar (Mohali)"]
}

def apply_mapping(df, state, mapping, label):
    total = 0
    for correct, wrongs in mapping.items():
        mask = (
            (df["state"] == state) &
            (df["district"].isin(wrongs))
        )
        count = mask.sum()
        df.loc[mask, "district"] = correct
        total += count
        if count > 0:
            print(f"✔ {label} → {correct} : {count} rows fixed")
    return total

total_fixes = 0
total_fixes += apply_mapping(enrol_df, STATE_NAME, DISTRICT_MAPPING, "Enrolment")
total_fixes += apply_mapping(demo_df,  STATE_NAME, DISTRICT_MAPPING, "Demographic")
total_fixes += apply_mapping(bio_df,   STATE_NAME, DISTRICT_MAPPING, "Biometric")

print(f"✅ Total fixes in {STATE_NAME}: {total_fixes}")


✔ Enrolment → Ferozepur : 289 rows fixed
✔ Enrolment → Sri Muktsar Sahib : 200 rows fixed
✔ Enrolment → Shaheed Bhagat Singh Nagar : 156 rows fixed
✔ Enrolment → Sahibzada Ajit Singh Nagar : 1201 rows fixed
✔ Demographic → Ferozepur : 1038 rows fixed
✔ Demographic → Sri Muktsar Sahib : 504 rows fixed
✔ Demographic → Shaheed Bhagat Singh Nagar : 1037 rows fixed
✔ Demographic → Sahibzada Ajit Singh Nagar : 2361 rows fixed
✔ Biometric → Ferozepur : 938 rows fixed
✔ Biometric → Sri Muktsar Sahib : 505 rows fixed
✔ Biometric → Shaheed Bhagat Singh Nagar : 1084 rows fixed
✔ Biometric → Sahibzada Ajit Singh Nagar : 2081 rows fixed
✅ Total fixes in Punjab: 11394


In [5]:
enrol_df.to_csv(CLEAN_DIR / "enrolment_clean.csv", index=False)
demo_df.to_csv(CLEAN_DIR / "demographic_clean.csv", index=False)
bio_df.to_csv(CLEAN_DIR / "biometric_clean.csv", index=False)

print("💾 All cleaned files saved successfully (overwritten)")


💾 All cleaned files saved successfully (overwritten)
